In [3]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [8]:
base_path = Path('data/')
list(base_path.glob("*/*/*.*"))


[PosixPath('data/tanpa_bola/take_269663/right_cam.jpg'),
 PosixPath('data/tanpa_bola/take_269663/left_cam.jpg'),
 PosixPath('data/tanpa_bola/take_441446/right_cam.jpg'),
 PosixPath('data/tanpa_bola/take_441446/New Text Document.txt'),
 PosixPath('data/tanpa_bola/take_441446/left_cam.jpg'),
 PosixPath('data/tanpa_bola/take_238608/right_cam.jpg'),
 PosixPath('data/tanpa_bola/take_238608/left_cam.jpg'),
 PosixPath('data/bola/take_700692/right_cam.jpg'),
 PosixPath('data/bola/take_700692/New Text Document.txt'),
 PosixPath('data/bola/take_700692/left_cam.jpg'),
 PosixPath('data/bola/take_086448/right_cam.jpg'),
 PosixPath('data/bola/take_086448/New Text Document.txt'),
 PosixPath('data/bola/take_086448/left_cam.jpg'),
 PosixPath('data/bola/take_152294/right_cam.jpg'),
 PosixPath('data/bola/take_152294/New Text Document.txt'),
 PosixPath('data/bola/take_152294/left_cam.jpg'),
 PosixPath('data/bola/take_921477/right_cam.jpg'),
 PosixPath('data/bola/take_921477/New Text Document.txt'),
 Posix